In [ ]:
!pip install pickle-mixin

  Preparing metadata (setup.py) ... done
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=5991 sha256=2182caa102f855db14ced5339c0e8e8c309408f95ab80080b53f16f6610e7b11
  Stored in directory: /root/.cache/pip/wheels/3e/c6/e9/d1b0a34e1efc6c3ec9c086623972c6de6317faddb2af0a619c
Successfully built pickle-mixin


**Importing required libraries:**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

**Gathering data:**

In [ ]:
movies = pd.read_csv( '/content/top10K-TMDB-movies.csv' )
print( movies.head() )

      id                        title                 genre original_language  \
0    278     The Shawshank Redemption           Drama,Crime                en   
1  19404  Dilwale Dulhania Le Jayenge  Comedy,Drama,Romance                hi   
2    238                The Godfather           Drama,Crime                en   
3    424             Schindler's List     Drama,History,War                en   
4    240       The Godfather: Part II           Drama,Crime                en   

                                            overview  popularity release_date  \
0  Framed in the 1940s for the double murder of h...      94.075   1994-09-23   
1  Raj is a rich, carefree, happy-go-lucky second...      25.408   1995-10-19   
2  Spanning the years 1945 to 1955, a chronicle o...      90.585   1972-03-14   
3  The true story of how businessman Oskar Schind...      44.761   1993-12-15   
4  In the continuing saga of the Corleone crime f...      57.749   1974-12-20   

   vote_average  vote_coun

**Data preprocessing:**

In [ ]:
print( movies.isnull().sum() )

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64


In [ ]:
print( movies.columns )

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')


In [ ]:
sub_dataset = movies[ [ 'id', 'title', 'genre', 'original_language', 'overview', 'popularity' ] ]
print( sub_dataset.head )

<bound method NDFrame.head of           id                                          title  \
0        278                       The Shawshank Redemption   
1      19404                    Dilwale Dulhania Le Jayenge   
2        238                                  The Godfather   
3        424                               Schindler's List   
4        240                         The Godfather: Part II   
...      ...                                            ...   
9995   10196                             The Last Airbender   
9996  331446                       Sharknado 3: Oh Hell No!   
9997   13995                                Captain America   
9998    2312  In the Name of the King: A Dungeon Siege Tale   
9999  455957                                         Domino   

                                                 genre original_language  \
0                                          Drama,Crime                en   
1                                 Comedy,Drama,Romance       

In [ ]:
sub_dataset[ 'description' ] = sub_dataset[ 'genre' ] + sub_dataset[ 'original_language' ] + sub_dataset[ 'overview' ]
sub_dataset = sub_dataset.drop( [ 'genre', 'original_language', 'overview' ], axis = 1 )
print( sub_dataset.head )

<bound method NDFrame.head of           id                                          title  popularity  \
0        278                       The Shawshank Redemption      94.075   
1      19404                    Dilwale Dulhania Le Jayenge      25.408   
2        238                                  The Godfather      90.585   
3        424                               Schindler's List      44.761   
4        240                         The Godfather: Part II      57.749   
...      ...                                            ...         ...   
9995   10196                             The Last Airbender      98.322   
9996  331446                       Sharknado 3: Oh Hell No!      12.490   
9997   13995                                Captain America      18.333   
9998    2312  In the Name of the King: A Dungeon Siege Tale      15.159   
9999  455957                                         Domino      16.482   

                                            description  
0     Drama

<ipython-input-7-2346c976cdb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_dataset[ 'description' ] = sub_dataset[ 'genre' ] + sub_dataset[ 'original_language' ] + sub_dataset[ 'overview' ]


In [ ]:
converter = CountVectorizer( max_features = 10000, stop_words = 'english' )
vectors = converter.fit_transform( sub_dataset[ 'description' ].values.astype( 'U' ) ).toarray()

In [ ]:
print( vectors )

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
print( vectors.shape )

(10000, 10000)


**Building recommendation system:**

In [ ]:
similar_vectors = cosine_similarity( vectors )
print( similar_vectors )

[[1.         0.06253054 0.05802589 ... 0.07963978 0.07597372 0.03481553]
 [0.06253054 1.         0.08980265 ... 0.         0.         0.        ]
 [0.05802589 0.08980265 1.         ... 0.02541643 0.03636965 0.        ]
 ...
 [0.07963978 0.         0.02541643 ... 1.         0.03327792 0.03049971]
 [0.07597372 0.         0.03636965 ... 0.03327792 1.         0.04364358]
 [0.03481553 0.         0.         ... 0.03049971 0.04364358 1.        ]]


**Testing:**

In [ ]:
sub_dataset[ sub_dataset[ 'title' ] == 'Dilwale Dulhania Le Jayenge' ].index[ 0 ]

1

In [ ]:
gap = sorted( list( enumerate( similar_vectors[ 1 ] ) ), reverse = True, key = lambda vectors : vectors[ 1 ] )
for _ in gap[ 1 : 6 ]:
  print( sub_dataset.iloc[ _[ 0 ] ].title )

The Cheetah Girls: One World
A Passage to India
Fatal Attraction
Noelle
Just Married


**Testing of recommendation based on user input:**

In [ ]:
def recommend_acc_to( movie ):
  index = sub_dataset[ sub_dataset[ 'title' ] == movie ].index[ 0 ]
  gap = sorted( list( enumerate( similar_vectors[ index ] ) ), reverse = True, key = lambda vectors : vectors[ 1 ] )
  for _ in gap[ 1 : 6 ]:
    print( sub_dataset.iloc[ _[ 0 ] ].title )

user_input = input( 'Enter a movie name:' )
recommend_acc_to( user_input )

Enter a movie name:Iron Man
Mazinger Z: Infinity
Justice League Dark
Iron Man 3
The Colony
Marvel One-Shot: Item 47


**Deploying models:**

In [ ]:
pickle.dump( sub_dataset, open( 'movies_list.pkl', 'wb' ) )
pickle.dump( similar_vectors, open( 'similar_movies.pkl', 'wb' ) )